In [4]:
import pandas as pd
import numpy as np
import torch
import time
from datasets import Dataset, DatasetDict

# Les paths de fichiers
path1 = r"C:\Users\Zakaria-Laptop\LLM-Models\NLP_TFIDF\DataBase\S08_question_answer_pairs.txt"
path2 = r"C:\Users\Zakaria-Laptop\LLM-Models\NLP_TFIDF\DataBase\S09_question_answer_pairs.txt"
path3 = r"C:\Users\Zakaria-Laptop\LLM-Models\NLP_TFIDF\DataBase\S10_question_answer_pairs.txt"


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.1.2 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "c:\Users\Zakaria-Laptop\LLM-Models\QuestionAnsweringTask\myenv\Lib\site-packages\ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "c:\Users\Zakaria-Laptop\LLM-Models\QuestionAnsweringTask\myenv\Lib\site-packages\traitlets\config\application.py", line 1075, in launch_instance
    app.start()
  File "c:\Users\Zakaria-Laptop\LLM-Models\QuestionAnsweringTask\myenv\Lib\site-packages\i

ModuleNotFoundError: No module named 'datasets'

In [29]:
def read_file(path: str) -> pd.DataFrame:
    """_summary_

    Args:
        path (str): _description_

    Returns:
        pd.DataFrame: _description_
    """    
    return pd.read_csv(path, sep="	")

data1 = read_file(path1)
data2 = read_file(path2)
# It's a file encoded with ISO
data3 = pd.read_csv(path3, sep="\t", encoding='ISO-8859-1')

data = pd.concat([data1, data2, data3])
data.iloc[6:9]


,ArticleTitle,Question,Answer,DifficultyFromQuestioner,DifficultyFromAnswerer,ArticleFile
6,Abraham_Lincoln,How many long was Lincoln's formal education?,18 months,medium,easy,S08_set3_a4
7,Abraham_Lincoln,How many long was Lincoln's formal education?,18 months.,medium,medium,S08_set3_a4
8,Abraham_Lincoln,When did Lincoln begin his political career?,1832,medium,easy,S08_set3_a4


## Preprocessing

In [3]:
copy_data = data.copy()
# Transformer en un Dataset Hugging Face
dataset = Dataset.from_pandas(copy_data)
dataset = dataset.remove_columns("__index_level_0__")
dataset

Dataset({
    features: ['ArticleTitle', 'Question', 'Answer', 'DifficultyFromQuestioner', 'DifficultyFromAnswerer', 'ArticleFile'],
    num_rows: 3998
})

In [8]:
from datasets import Dataset, DatasetDict

#Diviser les données en ensembles train, validation, test
train_df = copy_data.sample(frac=1, random_state=42)
remaining_df = copy_data.drop(train_df.index)
validation_df = remaining_df.sample(frac=0.25, random_state=42)
test_df = remaining_df.drop(validation_df.index)

# Créer les ensembles DataSet
train_dataset = Dataset.from_pandas(train_df)
validation_dataset = Dataset.from_pandas(validation_df)
test_dataset = Dataset.from_pandas(test_df)

# Créer les DatasetDict
dataset_dict = DatasetDict({
    "train": train_dataset,
    "validation": validation_dataset,
    "test": test_dataset,
})
dataset_dict = dataset_dict.remove_columns(['__index_level_0__', 'ArticleTitle', 'DifficultyFromQuestioner', 'DifficultyFromAnswerer', 'ArticleFile'])
data_finale = dataset_dict["train"]
print(data_finale)

Dataset({
    features: ['Question', 'Answer'],
    num_rows: 3998
})


## Théorie:

Pour créer un modèle qui fonctionne comme une FAQ, l'approche la plus courante consiste à utiliser des techniques de recherche de similarité de texte. Cela consiste à identifier quelle question de ton ensemble de données est la plus similaire à la question posée en entrée, puis à renvoyer la réponse correspondante.

## Les étapes:

__1 - Nettoyage et prétraitement des données :__

Nettoyer les questions et réponses (enlever les stop words, la ponctuation, etc.).

__2 - Vectorisation des questions :__

Convertir les questions en vecteurs numériques. Une méthode classique consiste à utiliser des techniques comme TF-IDF, Word2Vec ou des modèles plus avancés comme les embeddings BERT (Bidirectional Encoder Representations from Transformers) pour obtenir des représentations vectorielles des questions.

__3 - Calcul de la similarité :__

Utiliser une mesure de similarité, comme la similarité cosinus, pour comparer la question en entrée à toutes les questions de ton ensemble de données et identifier la question la plus proche.

__4 - Récupération de la réponse :__

Renvoyer la réponse correspondant à la question la plus similaire.

## Approche TF-IDF et Cos-similarité

In [7]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [20]:
# Supprimer les None de ma base de données vu que lors de l'application de vectorizer.fit_transform ça applique .lower() et avec des None ça retoure des erreurs
data = [item for item in data_finale if item["Question"] is not None and item["Question"].strip() != ""]

In [21]:
questions = [item["Question"] for item in data]
responses = [item["Answer"] for item in data]
print(f"len of questions is: {len(questions)} and len of answers is: {len(responses)}")

len of questions is: 3961 and len of answers is: 3961


In [22]:
# Etape1: Véctorisation les questions avec TF-IDF
vectorizer = TfidfVectorizer()
tf_idf_matrix = vectorizer.fit_transform(questions)


In [30]:
# Définir une fonction pour retourner la réponse basant sur le Cos-similarity

def get_answer(question: str) -> str:
    """La réponse avec TF-IDF fonction

    Args:
        question (str): la question d'entrée

    Returns:
        str: La réponse trouvée
    """    
    # Avoir le vector de la question
    question_tfidf = vectorizer.transform([question])

    # Appliquer le cos similarity avec ma matrice
    similarity = cosine_similarity(question_tfidf, tf_idf_matrix).flatten()

    # Trouver l'index de la question la plus similaire
    closest_question_index = np.argmax(similarity)

    # Retourner la réponse associe à cette question
    return responses[closest_question_index]

In [42]:
# Test

new_question = "How many cymbals are usually included in a drum kit?"
response = get_answer(new_question)
print(f"Question: {new_question}\nRéponse: {response}")

question2 = "How many long was Lincoln's formal education?"
response2 = get_answer(question2)
print(f"Question: {question2}\nRéponse: {response2}")

Question: How many cymbals are usually included in a drum kit?
Réponse: at least 3
Question: How many long was Lincoln's formal education?
Réponse: 18 months


## Amélioration avec BERT (pour de meilleures performances) :

Si tu veux une approche plus avancée avec un meilleur niveau de compréhension sémantique, tu peux utiliser un modèle de type BERT.

Voici comment utiliser BERT pour encoder les questions.



In [35]:
from transformers import BertTokenizer, BertModel
import torch

# Charger le model et le tokenizer
model_name = 'bert-base-uncased'
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertModel.from_pretrained(model_name)

c:\Users\Zakaria-Laptop\AppData\Local\Programs\Python\Python312\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [50]:
# Fonction pour obtenir les embeddings BERT
def get_bert_embeddings(text):
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True)
    with torch.no_grad():  # Désactiver le calcul des gradients pour une évaluation
        outputs = model(**inputs)
    # Prendre la moyenne des embeddings sur la séquence
    return outputs.last_hidden_state.mean(dim=1).squeeze().numpy()  # Utiliser squeeze pour réduire la dimension

# Encoder les questions
question_embeddings = np.array([get_bert_embeddings(q) for q in questions])

In [58]:
# Supposons que `question_embeddings` soit un tableau NumPy contenant les embeddings
np.save('question_embeddings.npy', question_embeddings)

# Load le fichier
#question_embeddings = np.load('question_embeddings.npy')

In [53]:
# Fonction pour trouver la réponse avec BERT embeddings
def find_answer_bert(new_question):
    new_question_embedding = get_bert_embeddings(new_question).reshape(1, -1)
    similarities = cosine_similarity(new_question_embedding, question_embeddings).flatten()
    closest_question_idx = np.argmax(similarities)
    return responses[closest_question_idx]


In [57]:
# Exemple d'utilisation avec BERT
new_question = "How many cymbals are usually included in a drum kit?"
response_bert = find_answer_bert(new_question)
print(f"Question: {new_question}\nRéponse: {response_bert}")

question2 = "How many long was Lincoln's formal education?"
response2 = find_answer_bert(question2)
print(f"Question: {question2}\nRéponse: {response2}")

Question: How many cymbals are usually included in a drum kit?
Réponse: at least 3
Question: How many long was Lincoln's formal education?
Réponse: 18 months


## Métriques d'évaluation

In [61]:
# Evaluer mes modèles Avec TFIDF
predicted_answers_tfidf = [get_answer(q) for q in questions]

y_true = [1 if ans == expected else 0 for ans, expected in zip(predicted_answers_tfidf, responses)]
y_pred = [1 if ans == expected else 0 for ans, expected in zip(predicted_answers_tfidf, responses)]

# Calcul des métriques
exact_matches = sum(p == e for p, e in zip(predicted_answers_tfidf, responses)) / len(responses) * 100

# Afficher les résultats
print(f"Exact Match: {exact_matches:.2f}%")

Exact Match: 70.74%


In [63]:
# Evaluer mes modèles avec BERT
predicted_answers_bert = [find_answer_bert(q) for q in questions]

y_true = [1 if ans == expected else 0 for ans, expected in zip(predicted_answers_bert, responses)]
y_pred = [1 if ans == expected else 0 for ans, expected in zip(predicted_answers_bert, responses)]

# Calcul des métriques
exact_matches = sum(p == e for p, e in zip(predicted_answers_bert, responses)) / len(responses) * 100

# Afficher les résultats
print(f"Exact Match: {exact_matches:.2f}%")

Exact Match: 70.76%


In [55]:
dash = "-".join("" for x in range(50))
print(f"{dash} To be continued {dash}")

------------------------------------------------- To be continued -------------------------------------------------
